In [2]:
import pandas as pd 
import numpy as np
from sklearn.metrics import precision_score ,recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline


In [3]:
train_data=pd.read_csv('train.csv')

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
X=train_data.drop("Survived",axis=1)
y=train_data["Survived"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [6]:
X_train.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            136
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          552
Embarked         1
dtype: int64

In [7]:
X_train = X_train.drop(labels=['Name','PassengerId','Cabin','Ticket'],axis=1)

In [8]:
X_train['Embarked'].value_counts()

Embarked
S    520
C    132
Q     59
Name: count, dtype: int64

In [9]:
X_train['Embarked']=X_train['Embarked'].fillna('S')

In [10]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         136
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

In [11]:
#Handle Missing Values
i = SimpleImputer(missing_values=np.nan, strategy='median')

In [12]:
i = i.fit(X_train[['Age']])

X_train[['Age']] = i.transform(X_train[['Age']])

X_test[['Age']]=i.transform(X_test[['Age']])




In [13]:
X_train.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [14]:
numerical_features = ['Age', 'Fare']

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])

In [15]:
X_train.head()

Pclass     Sex       Age  SibSp  Parch      Fare Embarked
553       3    male -0.554207      0      0 -0.505472        C
184       3  female -1.893163      0      2 -0.199307        S
695       2    male  1.677386      0      0 -0.375662        S
356       1  female -0.554207      0      1  0.482840        S
723       2    male  1.528613      0      0 -0.386006        S

In [16]:
train_data.drop_duplicates(inplace=True)

In [17]:

X_train = pd.get_dummies(X_train, columns=[ 'Embarked'])

In [18]:
features = ["Pclass", "Sex", "SibSp", "Parch"]
X_train = pd.get_dummies(X_train[features], drop_first=True)
X_test = pd.get_dummies(X_test[features], drop_first=True)


In [19]:
model_rf = RandomForestClassifier(min_samples_split=2, min_samples_leaf=1)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)
RF_precision = precision_score(y_test, y_pred_rf)
RF_recall = recall_score(y_test, y_pred_rf)


accuracy = accuracy_score(y_test, y_pred_rf)
print("Accuracy:", accuracy)
print(f"Precision: {RF_precision:.4f}")
print(f"Recall: {RF_recall:.4f}")


Accuracy: 0.7988826815642458
Precision: 0.8852
Recall: 0.6506


In [20]:

rf_param_grid = {
   'n_estimators':[50,75,100],
    'max_depth':[3,5,8],
    'min_samples_split':[0.1,0.15,0.2],
    
}
rf_grid_search = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv=5)
rf_grid_search.fit(X_train, y_train)
best_rf_model = rf_grid_search.best_estimator_

rf_predictions = best_rf_model.predict(X_test)
rf_precision = precision_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)

In [21]:
rf_precision = precision_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)
accuracy = accuracy_score(y_test, rf_predictions)
print("Accuracy:", accuracy)
print(f"Precision: {rf_precision:.4f}")
print(f"Recall: {rf_recall:.4f}")

Accuracy: 0.8212290502793296
Precision: 0.9180
Recall: 0.6747


In [22]:
logreg = LogisticRegression(C=8.0,random_state=40)
logreg.fit(X_train, y_train)
y_pred1 = logreg.predict(X_test)

log_precision1 = precision_score(y_test, y_pred1)
log_recall1 = recall_score(y_test, y_pred1)
accuracy1 = accuracy_score(y_test, y_pred1)

print("Accuracy:", accuracy1)
print(f"Precision: {log_precision1:.4f}")
print(f"Recall: {log_recall1:.4f}")

Accuracy: 0.8156424581005587
Precision: 0.8906
Recall: 0.6867


In [23]:
lr_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs'],
    

}
lr_grid_search = GridSearchCV(LogisticRegression(), lr_param_grid, cv=5, scoring='f1')
lr_grid_search.fit(X_train, y_train)
best_lr_model = lr_grid_search.best_estimator_
pred = best_lr_model.predict(X_test)

log_precision = precision_score(y_test, pred)
log_recall = recall_score(y_test, pred)
accuracy = accuracy_score(y_test, pred)

print("Accuracy:", accuracy)
print(f"Precision: {log_precision:.4f}")
print(f"Recall: {log_recall:.4f}")

Accuracy: 0.8156424581005587
Precision: 0.8906
Recall: 0.6867


In [24]:
# Compare precision and recall scores to choose the best model
if rf_precision > log_precision and rf_recall > log_recall:
    best_model = "Random Forest Classifier"
else:
    best_model = "Logistic Regression"

print("Best model based on precision and recall metrics:", best_model)

Best model based on precision and recall metrics: Logistic Regression


The chosen model balances precision and recall, optimizes hyperparameters, considers model flexibility, and generalizes well on unseen data.

In [25]:

# Perform cross-validation to evaluate the model's performance
cv_scores = cross_val_score(lr_grid_search, X_train, y_train, cv=5, scoring='accuracy')
avg_cv_score = np.mean(cv_scores)

print("Logistic Regression - Average Cross-Validation Score: {:.4f}".format(avg_cv_score))


Logistic Regression - Average Cross-Validation Score: 0.7893


In [26]:
# # Perform cross-validation to evaluate the model's performance
# cv_scores_rf = cross_val_score(rf_grid_search, X_train, y_train, cv=5, scoring='accuracy')
# avg_cv_score_rf = np.mean(cv_scores_rf)

# print("Random Forest - Average Cross-Validation Score: {:.4f}".format(avg_cv_score_rf))


In [27]:


# Create a pipeline with data preprocessing and logistic regression model
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Data preprocessing step
    ('log_reg', LogisticRegression())  # Logistic Regression model
])

# Define the hyperparameters grid for the pipeline
param_grid = {
    'scaler__with_mean': [True, False],  # Hyperparameters for the scaler
    'log_reg__C': [0.1, 1.0, 10.0],  # Hyperparameters for the logistic regression model
    'log_reg__penalty': ['l1', 'l2']
}

# Use GridSearchCV to tune the hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the best cross-validation score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Hyperparameters:", best_params)
print("Best Cross-Validation Score:", best_score)


Best Hyperparameters: {'log_reg__C': 0.1, 'log_reg__penalty': 'l2', 'scaler__with_mean': True}
Best Cross-Validation Score: 0.7892938047867626


c:\Users\Rama\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Rama\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Rama\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Rama\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 420, in fit
    self._final_estimator.fit(Xt,